In [9]:
import pandas as pd
import os
import glob
import requests
from datetime import datetime

# Retrieving and merging ACRA data #

In [15]:
def get_collection_metadata(collection_id):
    """
    Fetches metadata of a given collection from data.gov.sg and returns
    the collection's lastUpdatedAt timestamp as well as 
    a list of the IDs of the datasets it contains.
    """
    
    url = f"https://api-production.data.gov.sg/v2/public/api/collections/{collection_id}/metadata"
    headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return None  # stop early if API call failed

    data = response.json()

    last_updated = data['data']['collectionMetadata']['lastUpdatedAt']
    print("Last Updated:", last_updated)

    datasets = data['data']['collectionMetadata']['childDatasets']
    print(f"{len(datasets)} datasets found in collection.")

    return last_updated, datasets

In [11]:
def create_directory(last_updated): 
    """
    Create a directory reflecting the timestamp from 
    a data.gov.sg 'lastUpdatedAt' field
    """
    
    clean_ts = last_updated.replace(":", "-").replace("+", "-")
    dir_name = f"data_{clean_ts}"
    
    os.makedirs(dir_name, exist_ok=True)
    print(f"Directory created: {dir_name}")
    
    return dir_name

In [12]:
def get_dataset_name(dataset_id): 
    url = f"https://api-production.data.gov.sg/v2/public/api/datasets/{dataset_id}/metadata"
    response = requests.get(url)

    if response.status_code !=200: 
        print("Error:", response.status_code, response.text)
        return None  # stop early if API call failed
    
    data = response.json()

    name = data['data']['name']

    return name

In [13]:
def get_download_link(dataset_id):
    url = f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id}/initiate-download"
    response = requests.get(url)
    
    if response.status_code != 201:
        print("Error:", response.status_code, response.text)
        return None  # stop early if API call failed

    data = response.json()

    download_link = data['data']['url']
    
    return download_link

In [17]:
collection_id = 2
last_updated, datasets = get_collection_metadata(collection_id)
data_dir = create_directory(last_updated)
for dataset_id in datasets: 
    name = get_dataset_name(dataset_id)
    file_path = f"{data_dir}/{name.replace(" ", "").replace("('", "").replace("')", "")}.csv"
    response = requests.get(get_download_link(dataset_id))
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {file_path}")
    else:
        print('Failed to download file')

Last Updated: 2025-12-16T16:02:32+08:00
27 datasets found in collection.
Directory created: data_2025-12-16T16-02-32-08-00
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesW.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesU.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesZ.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesX.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesV.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesS.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesH.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesOthers.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesY.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInformationonCorporateEntitiesN.csv
Downloaded data_2025-12-16T16-02-32-08-00/ACRAInform

In [25]:
def combine_csvs(dir):
    '''Combines all CSV files in a given directory and exports them into a combined CSV. Assumes all CSV files follow same format'''

    os.chdir(dir)
    
    extension = 'csv'

    # Exit early if combined_csv.csv already exists
    if os.path.exists('combined_csv.csv'):
        combined_csv = pd.read_csv('combined_csv.csv')
        print("combined_csv.csv already exists. Skipping this step.")
        return combined_csv
        
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    print(f"CSV files found: {all_filenames}")
    
    if not all_filenames:
        print("No CSV files found.")
        return None
        
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
    print("CSVs successfully combined and exported to 'combined_csv.csv'")
    
    return combined_csv

In [26]:
df = combine_csvs(data_dir)

/var/folders/y7/2tm22kqn1vn2t_p4t87bt91h0000gn/T/ipykernel_26392/381204328.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_csv = pd.read_csv('combined_csv.csv')


combined_csv.csv already exists. Skipping this step.


# Basic data exploration, cleaning #

In [27]:
df.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,uen_of_audit_firm1,name_of_audit_firm1,uen_of_audit_firm2,name_of_audit_firm2,uen_of_audit_firm3,name_of_audit_firm3,uen_of_audit_firm4,name_of_audit_firm4,uen_of_audit_firm5,name_of_audit_firm5
0,00581600M,ACRA,88 FUNERAL PARLOUR,Sole Proprietorship/ Partnership,Sole-Proprietor,na,na,Cancelled,1974-10-25,1974-10-25,...,na,na,na,na,na,na,na,na,na,na
1,00811000W,ACRA,9 9 9,Sole Proprietorship/ Partnership,Sole-Proprietor,na,na,Terminated,1974-10-24,1974-10-24,...,na,na,na,na,na,na,na,na,na,na
2,02688300K,ACRA,20TH CENTURY BRASSIER CO,Sole Proprietorship/ Partnership,Partnership,na,na,Terminated,1974-11-09,1974-11-09,...,na,na,na,na,na,na,na,na,na,na
3,03312700W,ACRA,001 MANAGEMENT CONSULTANCY,Sole Proprietorship/ Partnership,Partnership,na,na,Ceased Registration,1974-12-13,1974-12-13,...,na,na,na,na,na,na,na,na,na,na
4,04331600W,ACRA,1 HOUR PHOTO,Sole Proprietorship/ Partnership,Sole-Proprietor,na,na,Terminated,1975-03-25,1975-03-25,...,na,na,na,na,na,na,na,na,na,na


In [28]:
df.columns

Index(['uen', 'issuance_agency_id', 'entity_name', 'entity_type_description',
       'business_constitution_description', 'company_type_description',
       'paf_constitution_description', 'entity_status_description',
       'registration_incorporation_date', 'uen_issue_date', 'address_type',
       'block', 'street_name', 'level_no', 'unit_no', 'building_name',
       'postal_code', 'other_address_line1', 'other_address_line2',
       'account_due_date', 'annual_return_date', 'primary_ssic_code',
       'primary_ssic_description', 'primary_user_described_activity',
       'secondary_ssic_code', 'secondary_ssic_description',
       'secondary_user_described_activity', 'no_of_officers',
       'former_entity_name1', 'former_entity_name2', 'former_entity_name3',
       'former_entity_name4', 'former_entity_name5', 'former_entity_name6',
       'former_entity_name7', 'former_entity_name8', 'former_entity_name9',
       'former_entity_name10', 'former_entity_name11', 'former_entity_name12'

In [29]:
cols_to_drop = [
    'uen', 'issuance_agency_id', 'entity_type_description', 'company_type_description',
    'paf_constitution_description', 'uen_issue_date', 
    'account_due_date', 'annual_return_date', 'no_of_officers',
    'former_entity_name1', 'former_entity_name2', 'former_entity_name3',
    'former_entity_name4', 'former_entity_name5', 'former_entity_name6',
    'former_entity_name7', 'former_entity_name8', 'former_entity_name9',
    'former_entity_name10', 'former_entity_name11', 'former_entity_name12',
    'former_entity_name13', 'former_entity_name14', 'former_entity_name15',
    'uen_of_audit_firm1', 'name_of_audit_firm1', 'uen_of_audit_firm2',
    'name_of_audit_firm2', 'uen_of_audit_firm3', 'name_of_audit_firm3',
    'uen_of_audit_firm4', 'name_of_audit_firm4', 'uen_of_audit_firm5',
    'name_of_audit_firm5', 'other_address_line1', 'other_address_line2'
]

df = df.drop(columns=cols_to_drop, errors="ignore")

In [30]:
df.apply(lambda col: col.dropna().map(type).unique())

entity_name                                         [<class 'str'>]
business_constitution_description                   [<class 'str'>]
entity_status_description                           [<class 'str'>]
registration_incorporation_date                     [<class 'str'>]
address_type                                        [<class 'str'>]
block                                               [<class 'str'>]
street_name                                         [<class 'str'>]
level_no                                            [<class 'str'>]
unit_no                                             [<class 'str'>]
building_name                                       [<class 'str'>]
postal_code                          [<class 'str'>, <class 'int'>]
primary_ssic_code                                   [<class 'int'>]
primary_ssic_description                            [<class 'str'>]
primary_user_described_activity                     [<class 'str'>]
secondary_ssic_code                             

In [31]:
df['postal_code'] = df['postal_code'].astype('str')

In [32]:
df.apply(lambda col: col.dropna().map(type).unique())

,entity_name,business_constitution_description,entity_status_description,registration_incorporation_date,address_type,block,street_name,level_no,unit_no,building_name,postal_code,primary_ssic_code,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [33]:
df['entity_status_description'].value_counts()

entity_status_description
Struck Off                                                        487821
Live Company                                                      454373
Terminated                                                        355163
Cancelled                                                         302621
Live                                                              152054
Cancelled (Non-Renewal)                                           130387
Ceased Registration                                               106356
Dissolved - Members Voluntary Winding Up                           34521
Gazetted To Be Struck Off                                           6665
Dissolved - Compulsory Winding Up (Insolvency)                      6090
Converted To LLP                                                    4138
Dissolved - Creditors Voluntary Winding Up                          3825
Amalgamated                                                         2514
In Liquidation - Compulso

In [34]:
df.shape

(2054168, 17)

In [35]:
df = df[df['entity_status_description'].str.contains('live', case=False, na=False)]

In [36]:
df.shape

(606520, 17)

In [37]:
df.head()

,entity_name,business_constitution_description,entity_status_description,registration_incorporation_date,address_type,block,street_name,level_no,unit_no,building_name,postal_code,primary_ssic_code,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity
9,1 OAK EVENT SERVICES,Sole-Proprietor,Live,1975-05-29,LOCAL,22,SIN MING LANE,06,76,MIDVIEW CITY,573969,82303,EVENT/CONCERT ORGANISERS,na,93120,EVENT/CONCERT ORGANISERS,na
15,3M SINGAPORE PTE. LTD.,na,Live Company,1966-07-27,LOCAL,10,ANG MO KIO STREET 65,03,01,TECHPOINT,569059,32909,na,na,64202,na,na
29,123 INTERNATIONAL MARINE SERVICE PTE. LTD.,na,Live Company,1973-12-12,LOCAL,111,NORTH BRIDGE ROAD,03,55,PENINSULA PLAZA,179098,50021,"SHIP&BOAT,TOWING,SALVAGING AND BROKING MANAGEMENT",na,na,"SHIP&BOAT,TOWING,SALVAGING AND BROKING MANAGEMENT",na
38,1ST CHOICE TRAVEL PTE LTD,na,Live Company,1977-08-29,LOCAL,31,MARINE CRESCENT,15,137,MARINE CRESCENT VILLE,440031,79102,TRAVEL AGENTS,na,na,TRAVEL AGENTS,na
51,3K'S PRIVATE LIMITED,na,Live Company,1979-08-10,LOCAL,3,MOUNT ELIZABETH,10,02,MOUNT ELIZABETH HOSPITAL,228510,86203,na,na,na,na,na


In [38]:
df['postal_code'].value_counts()

postal_code
409051    17418
079903     7264
79903      6956
179098     5087
608526     4571
          ...  
805427        1
787527        1
426200        1
499626        1
369376        1
Name: count, Length: 47922, dtype: int64